In [1]:
from functools import reduce
import csbgnpy.pd.io.sbgnml
import pypint
import multiprocessing
from IPython.display import display
import networkx as nx

This notebook has been executed using the docker image `pauleve/clockcycle:v0`

## Merged map

From [1 - Merge CYCLE and CLOCK](1%20-%20Merge%20CYCLE%20and%20CLOCK.ipynb)

In [2]:
merge_file = "generated/MERGE.sbgnml"
merge = csbgnpy.pd.io.sbgnml.read(merge_file)

In [3]:
clock = csbgnpy.pd.io.sbgnml.read("generated/CLOCK.sbgnml")
cycle = csbgnpy.pd.io.sbgnml.read("generated/CYCLE.sbgnml")
clock_ids = set([e.id for e in clock.entities])# + [p.id for p in clock.processes])
cycle_ids = set([e.id for e in cycle.entities])# + [p.id for p in cycle.processes])
clock_only = clock_ids - cycle_ids
cycle_only = cycle_ids - clock_ids

## Markers

From [2 - Markers.ipynb](2%20-%20Markers.ipynb):

In [4]:
%store -r cycle_markers
%store -r clock_markers

## Initial state

From [3 - Initial state](3%20-%20Initial%20state.ipynb)

In [5]:
%store -r ini

To ids:

In [6]:
ini = [e.id for e in ini]

### Conversion to internal identifiers

In [7]:
inirb = list(cycle_ids.intersection(ini))

In [8]:
def get_map(a):
    if a in clock_only:
        return "CLOCK"
    if a in cycle_only:
        return "CYCLE"
    return "MERGE"
def pretty_node(a):
    if a.startswith("epn_"):
        e = merge.get_entity(a, by_id=True)
    else:
        e = merge.get_process(a, by_id=True)
    return "{}::{}".format(get_map(a), str(e))

In [9]:
[pretty_node(e) for e in ini]

['CYCLE::Complex([SubMacromolecule([][@]DP1*)|SubMacromolecule([][@|@Lys|@Ser31|@Ser364|@Ser403]E2F1)|SubMacromolecule([][@|@|@|@]pRB*)][][]pRB*/E2F1/DP1*#Compartment([]nucleoplasm))',
 'CYCLE::Complex([SubMacromolecule([][]BMI1*)|SubMacromolecule([][]MEL-18*)|SubMacromolecule([][]PHC1)|SubMacromolecule([][]RING1)|SubMacromolecule([][]RYBP)][][]RYBP/RING1/BMI1*/ MEL-18*/PHC1#Compartment([]nucleoplasm))',
 'CYCLE::Macromolecule([][*@NF|@Thr172]CDK4#Compartment([]cytoplasm))',
 'CYCLE::Macromolecule([][@S317|@S345]CHEK1#Compartment([]nucleoplasm))',
 'MERGE::Macromolecule([][@S395]MDM2#Compartment([]cytoplasm))',
 'CYCLE::Macromolecule([][@Ser1981]ATM#Compartment([]nucleoplasm))',
 'CYCLE::Macromolecule([][@T145|@Thr57]p21Cip*#Compartment([]nucleoplasm))',
 'CYCLE::Macromolecule([][@Thr14|@Thr161|@Tyr15]CDK1#Compartment([]cytoplasm))',
 'CYCLE::Macromolecule([][@Thr157|@Thr187]p27Kip1*#Compartment([]nucleoplasm))',
 'CYCLE::Macromolecule([][@Thr160|@Tyr14|@Tyr15]CDK2#Compartment([]nucleo

In [10]:
an = pypint.load(merge_file, initial_state=ini)

In [11]:
an.save_as("generated/MERGE.an")

/notebook/generated/MERGE.an

In [12]:
def get_entity_id(spec):
    e = merge.get_entity(spec, by_string=True)
    if not e:
        raise Exception(f"No entity found for '{spec}'")
    return e.id

In [13]:
an_cycle_markers = dict([(p, [(get_entity_id(m),1) for m in ms]) \
                         for p,ms in cycle_markers.items()])
an_clock_markers = dict([(p, [(get_entity_id(m),1) for m in ms]) \
                         for p,ms in clock_markers.items()])

In [14]:
def pretty_mutations(m):
    return dict([(pretty_node(a), i) for (a,i) in sorted(m.items())])

def has_proc(m):
    for e in m:
        if e.startswith("proc_"):
            return True
    return False

def unpretty_node(a):
    name = "::".join(a.split("::")[1:])
    entity = merge.get_entity(name, by_string=True)
    return entity.id

def unpretty_mutations(m):
    return dict([(unpretty_node(a), i) for (a,i) in m.items()])

In [15]:
def oneshot_mutations_for_cut(an, markers, **kwargs):
    ms = []
    for marker in markers:
        ms += an.oneshot_mutations_for_cut(marker, **kwargs)
    return ms

In [16]:
def check_mutation_for_cut(an, markers, locks):
    for marker in markers:
        wt_reach = an.reachability(marker)
        mu_reach = an.lock(locks).reachability(marker)
        if wt_reach and not mu_reach:
            print("mutation is cutting {}".format(pretty_node(marker[0])))

In [17]:
def list_interesting(ms, interest):
    i2m = {}
    for mi, m in enumerate(ms):
        li = [(a,i) for (a,i) in m.items() if a in interest]
        arity = len(m)
        for (a,i) in li:
            a = pretty_node(a)
            if (a,i) not in i2m:
                i2m[(a,i)] = set()
            i2m[(a,i)].add(arity)
            #if arity not in i2m[(a,i)]:
            #    i2m[(a,i)][arity] = []
            #i2m[(a,i)][arity].append(mi)
    si2m = {}
    for k, v in sorted(i2m.items()):
        si2m[k] = v
    return si2m

In [18]:
ms = {}

## CYCLE markers

In [19]:
ms = an.oneshot_mutations_for_cut(an_cycle_markers["G0"], maxsize=3)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 3 automata. Use `maxsize` argument to change.

{('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))', 0): {3}}

In [20]:
ms = an.oneshot_mutations_for_cut(an_cycle_markers["earlyG1"], maxsize=3)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 3 automata. Use `maxsize` argument to change.

{('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))', 0): {1}}

In [21]:
ms = an.oneshot_mutations_for_cut(an_cycle_markers["lateG1"], maxsize=3)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 3 automata. Use `maxsize` argument to change.

{('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))', 0): {2, 3}}

In [22]:
ms = an.oneshot_mutations_for_cut(an_cycle_markers["earlyS"], maxsize=3)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 3 automata. Use `maxsize` argument to change.

{}

In [23]:
ms = an.oneshot_mutations_for_cut(an_cycle_markers["lateS"], maxsize=3)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 3 automata. Use `maxsize` argument to change.

{('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))', 0): {2, 3}}

In [24]:
ms = an.oneshot_mutations_for_cut(an_cycle_markers["G2"], maxsize=8)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 8 automata. Use `maxsize` argument to change.

{('CLOCK::Complex([SubMacromolecule([][@|@L538|@S17|P@S90|@T21]BMAL1)|SubMacromolecule([][]CK2α)][][]BMAL1/CK2a#Compartment([]nucleoplasm))',
  0): {3, 4},
 ('CLOCK::Complex([SubMacromolecule([][@|@S121|@S661]PER1)|SubMacromolecule([][@|@S280|@S71]CRY1)|SubMacromolecule([][@|@S558]CRY2)|SubMacromolecule([][@|Ac@L538|@S17|P@S90|@T21]BMAL1)|SubMacromolecule([][ADPr@]CLOCK)|SubMacromolecule([][Ac@|@|@S662]PER2)|SubMacromolecule([][]CSNK1δ)|SubMacromolecule([][]CSNK1ε)][][]PER/CRY/CSNK1/CLOCK/BMAL1#Compartment([]nucleoplasm))',
  1): {3, 4},
 ('CLOCK::Complex([SubMacromolecule([][@|Ac@L538|@S17|P@S90|@T21]BMAL1)|SubMacromolecule([][ADPr@]CLOCK)|SubMacromolecule([][]CK2α)][][]BMAL1/CLOCK/CK2α#Compartment([]nucleoplasm))',
  0): {3, 4},
 ('CLOCK::Complex([SubMacromolecule([][]MIZ1)|SubMacromolecule([][]MYC)][][]MYC/MIZ1#Compartment([]nucleoplasm))',
  1): {3, 4},
 ('CLOCK::Macromolecule([][@Ser472|@Ser53]WEE1#Compartment([]cytoplasm))',
  0): {3, 4},
 ('CLOCK::Macromolecule([][@|@L538|@S17|@

In [25]:
ms = an.oneshot_mutations_for_cut(an_cycle_markers["M"], maxsize=8)
list_interesting(ms, clock_ids)

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 8 automata. Use `maxsize` argument to change.

{('CLOCK::Complex([SubMacromolecule([][@|@L538|@S17|P@S90|@T21]BMAL1)|SubMacromolecule([][]CK2α)][][]BMAL1/CK2a#Compartment([]nucleoplasm))',
  0): {1},
 ('CLOCK::Complex([SubMacromolecule([][@|@S121|@S661]PER1)|SubMacromolecule([][@|@S280|@S71]CRY1)|SubMacromolecule([][@|@S558]CRY2)|SubMacromolecule([][@|Ac@L538|@S17|P@S90|@T21]BMAL1)|SubMacromolecule([][ADPr@]CLOCK)|SubMacromolecule([][Ac@|@|@S662]PER2)|SubMacromolecule([][]CSNK1δ)|SubMacromolecule([][]CSNK1ε)][][]PER/CRY/CSNK1/CLOCK/BMAL1#Compartment([]nucleoplasm))',
  1): {1},
 ('CLOCK::Complex([SubMacromolecule([][@|Ac@L538|@S17|P@S90|@T21]BMAL1)|SubMacromolecule([][ADPr@]CLOCK)|SubMacromolecule([][]CK2α)][][]BMAL1/CLOCK/CK2α#Compartment([]nucleoplasm))',
  0): {1},
 ('CLOCK::Complex([SubMacromolecule([][]MIZ1)|SubMacromolecule([][]MYC)][][]MYC/MIZ1#Compartment([]nucleoplasm))',
  1): {1},
 ('CLOCK::Macromolecule([][@Ser472|@Ser53]WEE1#Compartment([]cytoplasm))',
  0): {1},
 ('CLOCK::Macromolecule([][@|@L538|@S17|@S90|@T21]BMAL1#

## CLOCK markers

In [26]:
an_clock_markers.keys()

dict_keys(['RORG', 'SIRT1', 'BMAL1-CLOCK', 'PER12-CRY12'])

In [27]:
ms = an.oneshot_mutations_for_cut(an_clock_markers["RORG"], maxsize=8)
[pretty_mutations(m) for m in ms if cycle_ids.intersection(m)]

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 8 automata. Use `maxsize` argument to change.

[]

In [28]:
ms = an.oneshot_mutations_for_cut(an_clock_markers["SIRT1"], maxsize=8)
[pretty_mutations(m) for m in ms if cycle_ids.intersection(m)]

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 8 automata. Use `maxsize` argument to change.

[]

In [29]:
ms = an.oneshot_mutations_for_cut(an_clock_markers["BMAL1-CLOCK"], maxsize=8)
[pretty_mutations(m) for m in ms if cycle_ids.intersection(m)]

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 8 automata. Use `maxsize` argument to change.

[]

In [30]:
ms = an.oneshot_mutations_for_cut(an_clock_markers["PER12-CRY12"], maxsize=8)
[pretty_mutations(m) for m in ms if cycle_ids.intersection(m)]

This computation is an *under-approximation*: returned mutations are all valid, but they may be non-minimal, and some solutions may be missed.

Limiting solutions to mutations of at most 8 automata. Use `maxsize` argument to change.

[]